In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('/kaggle/input/credit-card-customer-data/Credit Card Customer Data.csv')

## EDA

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
data.isnull().sum() #no missing value has been fouond

In [ ]:
data.drop(columns=['Sl_No', 'Customer Key'],inplace=True) # drop unnecessory column

### UNIVARIATE ANALYSIS

In [ ]:
i=1
plt.figure(figsize=(20,20))
for j in data.columns:
    plt.subplot(4,3,i)
    sns.boxplot(data=data,x=j)
    i+=1

#### Here we have some outlier but we dont remove it due to the loss of data

In [ ]:
i=1
plt.figure(figsize=(20,20))
for j in data.columns:
    plt.subplot(4,3,i)
    sns.histplot(data=data,x=data[j],kde=True)
    i+=1

### BIVARIATE ANALYSIS

In [ ]:
data.corr()

sns.heatmap(data.corr(),cmap='gist_earth_r',linewidths=1,annot=True)

#### 1- avg_credit_limit and total_credit_cards has the high +ve correlation of 0.61..                                                                                                                                                  2- avg_credit_limit and total_visits_online has the high +ve correlation of 0.55..                                                                                                                                                                                                                                                                  3-total_credit_card and total_calls_made has the high -ve correlation of -0.65....                                                                                                                                                    4-total_visit_bank and total_visit_online has high -ve correlation of -0.55....                                                                                                                                                    5-total_calls_made and total_visit_bank has high -ve correlation of -0.51.....

In [ ]:
sns.pairplot(data)

## K-MEAN CLUSTERING

In [ ]:
sc=StandardScaler()# scaling of data
scaled=sc.fit_transform(data)
scaled_data=pd.DataFrame(scaled,columns=data.columns)


In [ ]:
wcss=[]
for i in range(1,13):
    km=KMeans(n_clusters=i)
    km.fit(scaled_data)
    wcss.append(km.inertia_)

In [ ]:
slh_score=[]
for i in range(2,13):
    km=KMeans(n_clusters=i)
    km.fit(data)
    slh_score.append(silhouette_score(scaled_data,km.labels_))

In [ ]:
plt.plot(range(1,13),wcss)
plt.ylabel('wcss')
plt.xlabel('k-value')

In [ ]:
plt.plot(range(2,13),slh_score)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.grid(True)

#### Here we have choose k- value such that wcss is minimum and silihoutte coefficint is more so we choose k=3

In [ ]:
km1=KMeans(n_clusters=3)
km1.fit(scaled_data)
y=km1.predict(scaled_data)
data['label']=y

In [ ]:
sns.pairplot(data,hue='label',palette='bright')

In [ ]:
center=km1.cluster_centers_# centre of clustering
center

In [ ]:
Kmean_silihoute=silhouette_score(scaled_data,km1.labels_)
Kmean_silihoute

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=data,y='Avg_Credit_Limit',x='Total_Credit_Cards',hue='label',palette='bright')
plt.show()

#### segment-0:total_credit_card[4-6] and avg_credit_limit(0-75000)----    
#### segment-1:total_credit_card(0-3) and avg_credit_limit(0-25000)----
#### segment-2:total_credit_card(6-10) and avg_credit_limit>75000 ----

#### we can observe that higher the avg_credit_limit higher is total_credit_card aquired by customer










In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=data,y='Avg_Credit_Limit',x='Total_visits_online',hue='label',palette='bright')
plt.show()

#### segment-0:total_visit_online<=2 and avg_credit_limit<=100000
#### segment-1:total_visit_online[3-5] and avg_credit_limit<=25000
#### segment-2: total_visit_online[6-15] and avg_credit_limit>100000
#### we can see that greater the avg_credit_limit,customer visit more frequently online(6-15)

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=data,x='Total_calls_made',y='Total_Credit_Cards',hue='label',palette='bright')

#### segment-0:total_call_made[0-4] and total_credit_card[4-6]
#### segment-1:total_call_made[4-10] and total_credit_card[0-4]
#### segment-2:total_call_made[0-2] and total_credit_card[7-10]
#### less the total call made greater the total cedit card

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=data,x='Total_visits_online',y='Total_Credit_Cards',hue='label',palette='bright')

#### here we can also see that person having credit card (6-10) more frequently he visits online[6-15]

In [ ]:
import scipy.cluster.hierarchy as shc

In [ ]:
plt.figure(figsize=(10,8))
dend=shc.dendrogram(shc.linkage(scaled_data,method='ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
h_cluster=AgglomerativeClustering(n_clusters=3,affinity='euclidean',linkage='ward')

In [ ]:
y_h=h_cluster.fit_predict(scaled_data)

In [ ]:
data['H_label']=y_h

In [ ]:
H_silihoute=silhouette_score(scaled_data,h_cluster.labels_)
H_silihoute

In [ ]:
sns.pairplot(data=data,hue='H_label',palette='bright')

#### Silhouette Coefficient:
#### Silhouette Coefficient or silhouette score is a metric used to calculate the goodness of a clustering technique. Its value ranges from -1 to 1.
#### 1: Means clusters are well apart from each other and clearly distinguished.
#### 0: Means clusters are indifferent, or we can say that the distance between clusters is not significant.
#### -1: Means clusters are assigned in the wrong way.

In [ ]:
print(Kmean_silihoute,H_silihoute)# silihoute score for both clustering k-mean and H-clustering